In [ ]:
import json
import io
from datetime import datetime
import sys
import os.path
from functools import reduce
import openai

openai.api_key = os.environ['CHAT_GPT_TOKEN']
datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [ ]:
def get_summary(text: str, max_tokens, temperature=0.9, engine="davinci") -> str:
    max_total_tokens = 2049
    text = f"Please summarize the following text in {max_tokens} tokens: \n\n[{text}]"
    chunks = [text[i:i+max_total_tokens] for i in range(0, len(text), max_total_tokens)]
    max_summary_tokens = int(max_tokens / len(chunks))
    print(f"max_summary_tokens: {max_summary_tokens}")
    summaries = []
    for chunk in chunks:
        print(f"chunk: {chunk}")
        response = openai.Completion.create(
            engine=engine,
            prompt=chunk,
            max_tokens=max_summary_tokens,
            temperature=temperature,
            top_p=1
        )
        print("response: ", response)
        summaries.append(response.choices[0].text)
    summary = " ".join(summaries)
    return summary

print(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

In [ ]:
text = "This is a test of the emergency broadcast system. This is only a test. If this had been an actual emergency, you would have been instructed to do something. This concludes this test of the emergency broadcast system."
get_summary(text, 100, temperature=0.9, engine="text-davinci-003")

In [42]:
def split_text(text: str, max_chars: int=2049) -> list:

    words = text.split()
    groups = []
    current_group = ""

    for word in words:
        if len(current_group) < max_chars:
            current_group += " " + word
        else:
            groups.append(current_group[1:])
            current_group = " " + word

    return groups

text = "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed tincidunt ornare ipsum, a pretium orci malesuada eu. Suspendisse non sollicitudin velit, at ullamcorper urna. Donec et enim dapibus, elementum lectus et, tincidunt purus. Vivamus ultricies, turpis ut elementum interdum, purus dolor consectetur urna, at euismod nulla libero non purus. Nullam ac dui id ex imperdiet aliquam. Maecenas in luctus massa. Nulla facilisi. Fusce venenatis interdum tortor non laoreet. Donec lacinia ex quis vehicula elementum. Integer vel pretium velit. Morbi scelerisque tellus et dolor porttitor scelerisque. Suspendisse euismod, sem sit amet vehicula interdum, erat diam rutrum elit, eu tincidunt mauris justo sed tellus."

groups = split_text(text, 10)
print(groups)


['Lorem ipsum', 'dolor sit', 'amet, consectetur', 'adipiscing', 'elit. Sed', 'tincidunt', 'ornare ipsum,', 'a pretium', 'orci malesuada', 'eu. Suspendisse', 'non sollicitudin', 'velit, at', 'ullamcorper', 'urna. Donec', 'et enim dapibus,', 'elementum', 'lectus et,', 'tincidunt', 'purus. Vivamus', 'ultricies,', 'turpis ut', 'elementum', 'interdum,', 'purus dolor', 'consectetur', 'urna, at euismod', 'nulla libero', 'non purus.', 'Nullam ac', 'dui id ex', 'imperdiet', 'aliquam. Maecenas', 'in luctus', 'massa. Nulla', 'facilisi.', 'Fusce venenatis', 'interdum tortor', 'non laoreet.', 'Donec lacinia', 'ex quis vehicula', 'elementum.', 'Integer vel', 'pretium velit.', 'Morbi scelerisque', 'tellus et', 'dolor porttitor', 'scelerisque.', 'Suspendisse', 'euismod, sem', 'sit amet vehicula', 'interdum,', 'erat diam', 'rutrum elit,', 'eu tincidunt', 'mauris justo']


In [19]:
import re

def split_text(text: str) -> list:
    # Split the text into sentences
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)

    # Split each sentence into words and return a nested list
    return [sentence.split() for sentence in sentences]

text = "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed tincidunt ornare ipsum, a pretium orci malesuada eu. Suspendisse non sollicitudin velit, at ullamcorper urna. Donec et enim dapibus, elementum lectus et, tincidunt purus. Vivamus ultricies, turpis ut elementum interdum, purus dolor consectetur urna, at euismod nulla libero non purus. Nullam ac dui id ex imperdiet aliquam. Maecenas in luctus massa. Nulla facilisi. Fusce venenatis interdum tortor non laoreet. Donec lacinia ex quis vehicula elementum. Integer vel pretium velit. Morbi scelerisque tellus et dolor porttitor scelerisque. Suspendisse euismod, sem sit amet vehicula interdum, erat diam rutrum elit, eu tincidunt mauris justo sed tellus."

sentences_and_words = split_text(text)
print(sentences_and_words)


[['Lorem', 'ipsum', 'dolor', 'sit', 'amet,', 'consectetur', 'adipiscing', 'elit.'], ['Sed', 'tincidunt', 'ornare', 'ipsum,', 'a', 'pretium', 'orci', 'malesuada', 'eu.'], ['Suspendisse', 'non', 'sollicitudin', 'velit,', 'at', 'ullamcorper', 'urna.'], ['Donec', 'et', 'enim', 'dapibus,', 'elementum', 'lectus', 'et,', 'tincidunt', 'purus.'], ['Vivamus', 'ultricies,', 'turpis', 'ut', 'elementum', 'interdum,', 'purus', 'dolor', 'consectetur', 'urna,', 'at', 'euismod', 'nulla', 'libero', 'non', 'purus.'], ['Nullam', 'ac', 'dui', 'id', 'ex', 'imperdiet', 'aliquam.'], ['Maecenas', 'in', 'luctus', 'massa.'], ['Nulla', 'facilisi.'], ['Fusce', 'venenatis', 'interdum', 'tortor', 'non', 'laoreet.'], ['Donec', 'lacinia', 'ex', 'quis', 'vehicula', 'elementum.'], ['Integer', 'vel', 'pretium', 'velit.'], ['Morbi', 'scelerisque', 'tellus', 'et', 'dolor', 'porttitor', 'scelerisque.'], ['Suspendisse', 'euismod,', 'sem', 'sit', 'amet', 'vehicula', 'interdum,', 'erat', 'diam', 'rutrum', 'elit,', 'eu', 'tinci